# Toy example 1

## Objects

- Set of students, schools, routes, and types: 
$S = \{s_1,...,s_{n_S}\}, C = \{c_1,...,c_{n_C}\}, R = \{r_1,...,r_{n_R}\}$.
- Routes have a capacity of 1: $q_r = 1$.
- Each route serves one school.

Routes are effectively taxi services

- Students can select either a school, or school + transport bundle.
- Routes are only available in selected areas (only feasible for a subset of students $S_r \in S$).
- Student preferences over schools are strict and complete.
- 2 school priority brackets given by:
    - Students *within* a certain distance of a school selecting that school **without** transport AND students *within* a selected area served by a route, selecting a school **with** transport are placed in the same priority bracket; all other students placed in the bracket below this.
- Ties broken randomly within priority brackets.

## Algorithm

1. All objects unassigned. $\mu = \emptyset$
2. While some student $s_i$ is unmatched:
    1. $s_i$ proposes to the first object $a_j$ on their list; $\mu = \mu \cup \{s_i, a_j\}$.
        - Each object $a$ contains exactly one school.
        - Each route serves only one school.
        - Each route only serves one neighbourhood.
    2. If the school in the object is oversubscribed:
        - The lowest priority student $s_k$ in the priority list for the school in $a_j$ is released from the matching: $\mu = \mu \backslash \{s_k, a_j\}$ and released from the route.
        - All students lower in priority than s_k are removed from the priority list of the school in $a_j$ (and all of those students remove objects that contain the school in $a_j$ from their preference lists).
    3. If the route in the object has reached capacity:
        - The lowest priority student $s_k$ competing for the route in the priority list for the school in $a_j$ is released from the matching: $\mu = \mu \backslash \{s_k, a_j\}$.

# Simple matching code

In [ ]:
from itertools import permutations,product

# User-defined number of participants
n_students = 3
n_colleges = 2
n_routes = 1

# User-defined college and route capacities
c_cap = 1
r_cap = 1

# Lists of all participants and routes
student_names = [f"s_{i+1}" for i in range(n_students)];all_student_names = permutations(student_names)
college_names = [f"c_{i+1}" for i in range(n_colleges)];all_college_names = permutations(college_names)
route_names = [f"r_{i+1}" for i in range(n_routes)];all_route_names = permutations(route_names)

# Capacities of colleges and routes
c_caps = {c:c_cap for c in college_names};print(c_caps)
r_caps = {r:r_cap for r in route_names};print(r_caps)

# Student preferences
student_preferences = {s:[c for c in college_names[:]] for s in student_names[:]} #colon makes shallow copy to avoid mutation of names

# College priorities
college_priorities = {c:[s for s in student_names[:]] for c in college_names[:]} #colon makes shallow copy to avoid mutation of names

print(f"student preferences:\n{student_preferences}")
print(f"college priorities:\n{college_priorities}")

# Deferred Acceptance algorithm
free = [s for s in student_names]
matching = {}
unassigned = []

while free:
    print("\n")
    print("list of free students:",free)
    print(f"college capacities are {c_caps}")
    print("-"*50)

    # assign first student in the list of free students
    s = free.pop(0) # argument selects the first student in the list
    print("assigning student",s)
    print(s,"has preferences:",student_preferences[s])

    # identify first student's top college
    c = student_preferences[s][0]
    print(f"{s}'s top college is {c}")
    print("-"*50)
    
    # tentative matching
    matching.update({s:c})
    print("tentative matching:",matching)
    print("-"*50)
    
    # handling college oversubscription following priority lists
    if c_caps[c] == 0:
        print(f"{c} has reached capacity!")

        ranking = {college:rank for rank,college in enumerate(college_priorities[c])}
        lowest_priority = max(
            (item for item in matching if item in ranking),
            key=lambda item: ranking[item],
            default=None)
        print(f"lowest priority student is {lowest_priority}")

        matching.pop(lowest_priority)
        print(f"{lowest_priority} unmatched with {c}")

        student_preferences[lowest_priority].remove(c)
        print(f"reduced preference lists: {student_preferences}")

        college_priorities[c].remove(lowest_priority)
        print(f"reduced priority lists {college_priorities}")

        if student_preferences[lowest_priority]:
            free.append(lowest_priority)
        else:
            unassigned.append(s)
            print(f"{lowest_priority} has emptied their preference list and is unassigned")
        
        print("new list of free students:",free)
    else:
        c_caps[student_preferences[s][0]] -= 1
        print("college capacities are:",c_caps)
    
    print("current matching is:",matching)
    if unassigned: print(f"currently unassigned students: {unassigned}")
    #break

print("\n")
print(f"final matching: {matching}")
print(f"unassigned student(s): {unassigned}")

{'c_1': 1, 'c_2': 1}
{'r_1': 1}
student preferences:
{'s_1': ['c_1', 'c_2'], 's_2': ['c_1', 'c_2'], 's_3': ['c_1', 'c_2']}
college priorities:
{'c_1': ['s_1', 's_2', 's_3'], 'c_2': ['s_1', 's_2', 's_3']}


list of free students: ['s_1', 's_2', 's_3']
college capacities are {'c_1': 1, 'c_2': 1}
--------------------------------------------------
assigning student s_1
s_1 has preferences: ['c_1', 'c_2']
s_1's top college is c_1
--------------------------------------------------
tentative matching: {'s_1': 'c_1'}
--------------------------------------------------
college capacities are: {'c_1': 0, 'c_2': 1}
current matching is: {'s_1': 'c_1'}


list of free students: ['s_2', 's_3']
college capacities are {'c_1': 0, 'c_2': 1}
--------------------------------------------------
assigning student s_2
s_2 has preferences: ['c_1', 'c_2']
s_2's top college is c_1
--------------------------------------------------
tentative matching: {'s_1': 'c_1', 's_2': 'c_1'}
-----------------------------------

# Routes included

In [13]:
class Student:
    def __init__(self,name,preferences):
        self.name = name
        self.preferences = preferences

    def __repr__(self):
        return self.name


class College:
    def __init__(self,name,capacity,priorities,routes):
        self.name = name
        self.capacity = capacity
        self.priorities = priorities
        self.routes = routes

    def __repr__(self):
        return self.name


class Route:
    def __init__(self,name,capacity,college):
        self.name = name
        self.capacity = capacity
        self.college = college

    def __repr__(self):
        return self.name

In [29]:
from random import sample,randint
from itertools import permutations,product

# User-defined number of participants
n_students = 5
n_colleges = 2
n_routes = 1

# User-defined college and route capacities
c_cap = 2
r_cap = 1

# Lists of all participants and routes
student_names = [f"s_{i+1}" for i in range(n_students)]
print(f"list of student names: {student_names}")
college_names = [f"c_{i+1}" for i in range(n_colleges)]
print(f"list of college names: {college_names}")
route_names = [f"r_{i+1}" for i in range(n_routes)]
print(f"list of route names: {route_names}")
print("-"*50)

# Capacities of colleges and routes
c_caps = {c:c_cap for c in college_names};print(f"college capacities: {c_caps}")
r_caps = {r:r_cap for r in route_names};print(f"route capacities: {r_caps}")
print("-"*50)

# Route services
r_serves = (dict(zip(route_names[:],college_names[:])),dict(zip(college_names[:],route_names[:])))
print(f"route services: {r_serves}")

# Students served by routes
s_served = {s:route_names[0] for s in student_names[3:]}
#s_served = {s:route_names[randint(0,len(route_names)-1)] for s in sample(student_names,randint(0,len(student_names)))}
print(f"served students:{s_served}")
print("-"*50)

# Student preferences
student_preferences = {s:college_names[:] for s in student_names[:]} #colon makes shallow copy to avoid mutation of name

# College priorities
college_priorities = {c:student_names[:] for c in college_names[:]} #colon makes shallow copy to avoid mutation of names

# Add routes to preferences and priorities
for s in student_names:
    if s in s_served.keys():
        student_preferences[s].insert(0,(s_served[s],r_serves[0][s_served[s]]))

    for c in college_names:
        if (c in r_serves[1].keys()) and ((r_serves[1][c],c) in student_preferences[s]):
            college_priorities[c].insert(0,((r_serves[1][c],s)))

print(f"student preferences:\n{student_preferences}")
print(f"college priorities:\n{college_priorities}")
print("-"*50)

# Assign class objects
students = {s:Student(s,student_preferences[s]) for s in student_names}
print(f"dictionary of student objects: {students}")

colleges = {c:College(c,c_caps[c],college_priorities[c],r_serves[1][c] if c in r_serves[1].keys() else None) for c in college_names}
print(f"dictionary of college objects: {colleges}")

routes = {r:Route(r,r_caps[r],r_serves[0][r]) for r in route_names}
print(f"dictionary of route objects: {routes}")
print("-"*50)

# Deferred Acceptance algorithm
free = [s for s in student_names]
matching = {}
unassigned = []

while free:
    print("\n")
    print("list of free students:",free)
    print(f"college capacities are {c_caps}")
    print("-"*50)

    # assign first student in the list of free students
    s = free.pop(0) # argument selects the first student in the list
    print("assigning student",s)
    print(s,"has preferences:",students[s].preferences)

    # identify first student's top preference
    p = students[s].preferences[0]
    print(f"{s}'s top preference is {p}")
    print("-"*50)

    # tentative matching
    matching.update({s:p})
    print("tentative matching:",matching)
    print("-"*50)
    
    # handling college oversubscription following priority lists
    if p in college_names and type(p) is not tuple: c = p ;print(c,p)
    elif any(item in p for item in college_names): r,c = p[0],p[1] ;print(r,c,p)

    if colleges[c].capacity == 0:
        print(f"{c} has reached capacity!")

        ranking = {college:rank for rank,college in enumerate(colleges[c].priorities)}
        lowest_priority = max(
            (item for item in matching if item in ranking),
            key=lambda item: ranking[item],
            default=None)
        print(f"lowest priority student is {lowest_priority}")

        matching.pop(lowest_priority)
        print(f"{lowest_priority} unmatched with {p}")

        students[lowest_priority].preferences.remove(p)
        print(f"reduced preference list for {lowest_priority}: {students[lowest_priority].preferences}")

        colleges[c].priorities.remove(lowest_priority)
        print(f"reduced priority list for {c}: {colleges[c].priorities}")

        if students[lowest_priority].preferences:
            free.append(lowest_priority)
        else:
            unassigned.append(s)
            print(f"{lowest_priority} has emptied their preference list and is unassigned")
        
        print("new list of free students:",free)
    else:
        colleges[c].capacity -= 1
        print("college capacities are:",colleges)
    
    print("current matching is:",matching)
    if unassigned: print(f"currently unassigned students: {unassigned}")
    #break

print("\n")
print(f"final matching: {matching}")
print(f"unassigned student(s): {unassigned}")

list of student names: ['s_1', 's_2', 's_3', 's_4', 's_5']
list of college names: ['c_1', 'c_2']
list of route names: ['r_1']
--------------------------------------------------
college capacities: {'c_1': 2, 'c_2': 2}
route capacities: {'r_1': 1}
--------------------------------------------------
route services: ({'r_1': 'c_1'}, {'c_1': 'r_1'})
served students:{'s_4': 'r_1', 's_5': 'r_1'}
--------------------------------------------------
student preferences:
{'s_1': ['c_1', 'c_2'], 's_2': ['c_1', 'c_2'], 's_3': ['c_1', 'c_2'], 's_4': [('r_1', 'c_1'), 'c_1', 'c_2'], 's_5': [('r_1', 'c_1'), 'c_1', 'c_2']}
college priorities:
{'c_1': [('r_1', 's_5'), ('r_1', 's_4'), 's_1', 's_2', 's_3', 's_4', 's_5'], 'c_2': ['s_1', 's_2', 's_3', 's_4', 's_5']}
--------------------------------------------------
dictionary of student objects: {'s_1': s_1, 's_2': s_2, 's_3': s_3, 's_4': s_4, 's_5': s_5}
dictionary of college objects: {'c_1': c_1, 'c_2': c_2}
dictionary of route objects: {'r_1': r_1}
------

KeyError: None

In [1]:
from itertools import permutations,product

# User-defined number of participants
n_students = 3
n_colleges = 2
n_routes = 1

# User-defined college and route capacities
c_cap = 1
r_cap = 1

# Lists of all participants and routes
student_names = [f"s_{i+1}" for i in range(n_students)];all_student_names = permutations(student_names)
college_names = [f"c_{i+1}" for i in range(n_colleges)];all_college_names = permutations(college_names)
route_names = [f"r_{i+1}" for i in range(n_routes)];all_route_names = permutations(route_names)

# Capacities of colleges and routes
c_caps = {c:c_cap for c in college_names};print(c_caps)
r_caps = {r:r_cap for r in route_names};print(r_caps)

# Student preferences
student_preferences = {s:college_names[:] for s in student_names[:]}
#student_preferences = {s:{c:rank for rank,c in enumerate(college_names[:])} for s in student_names[:]}

# College priorities
college_priorities = {c:student_names[:] for c in college_names[:]}

print(f"student preferences:\n{student_preferences}")
print(f"college priorities:\n{college_priorities}")

{'c_1': 1, 'c_2': 1}
{'r_1': 1}
student preferences:
{'s_1': ['c_1', 'c_2'], 's_2': ['c_1', 'c_2'], 's_3': ['c_1', 'c_2']}
college priorities:
{'c_1': ['s_1', 's_2', 's_3'], 'c_2': ['s_1', 's_2', 's_3']}


### Example 1

- 2 students, schools and 2 routes.
- Student preferences:
    - $S_1: c_1 \succ \{r_1,c_1\} \succ \{r_2,c_2\} \succ c_2$
    - $S_2: \{r_1,c_1\} \succ \{r_2,c_2\} \succ c_1 \succ c_2$
- School priorities:
    - $C_1: S_1 \succ S_2$
    - $C_2: S_1 \succ S_2$
- Matchings:
    - S_1: C_1
    - S_2: C_2

### Example 2

- 3 students, 2 schools and 1 routes.
- Student preferences:
    - $S_1: c_1 \succ \{r_1,c_1\} \succ \{r_2,c_2\} \succ c_2$
    - $S_2: \{r_1,c_1\} \succ \{r_2,c_2\} \succ c_1 \succ c_2$
    - $S_3: \{r_1,c_1\} \succ \{r_2,c_2\} \succ c_1 \succ c_2$
- School priorities:
    - $C_1: S_1 \succ S_2 \succ S_3$
    - $C_2: S_1 \succ S_3 \succ S_2$
- Matchings:
    - S_1: C_1
    - S_2: C_2